# Le jeu Palworld et analyse des données

## Présentation du jeu Palworld

Palworld est un jeu de survie en monde ouvert développé par Pocket Pair, sorti en janvier 2024. Ce jeu combine des éléments de survie, de crafting, de collection de créatures et de construction de base. Il se déroule dans un monde ouvert peuplé de créatures appelées "Pals" que les joueurs peuvent capturer, élever et utiliser pour diverses tâches.

Les aspects clés du jeu incluent:

| Aspect |
| - |
| La capture et la collection de Pals (similaire à Pokémon) |
| L'utilisation des Pals pour combattre |
| L'exploitation des Pals pour des tâches comme la construction, l'agriculture, et le travail en usine |
| La construction et le développement d'un campement |
| L'exploration d'un monde ouvert avec différentes zones et biomes |

Le jeu a connu un immense succès dès sa sortie en raison de son concept innovant mêlant des mécaniques de jeux populaires comme Pokémon et Minecraft, mais avec une approche plus mature et parfois controversée.


## 📁 Analyse des Fichiers CSV et leur Rôle dans le Jeu

### Vue d'ensemble des Données Palworld

Sur la base des descriptions fournies, voici une analyse de chaque fichier de données et son importance dans le jeu :

### 1.  Palworld_Data--Palu combat attribute table.csv

**Contenu :**
- Statistiques de base (santé, attaque, défense, vitesse)
- Types élémentaires (Feu, Eau, Électricité, etc.)
- Compétences de combat spécifiques

**Rôle dans le jeu :** Ces données déterminent l'efficacité d'un Pal au combat. Les joueurs utilisent ces informations pour construire des équipes équilibrées et stratégiques pour les combats contre d'autres Pals ou les boss.

### 2.  Palworld_Data--Palu refresh level.csv

**Contenu :** Informations sur les niveaux auxquels les Pals apparaissent dans différentes zones du jeu.

**Rôle dans le jeu :** Cette information est cruciale pour les joueurs qui cherchent à capturer des Pals spécifiques. Elle permet de savoir où trouver certains Pals et à quel niveau ils apparaîtront, ce qui affecte leur difficulté de capture et leurs statistiques initiales.

### 3. Palworld_Data-Palu Job Skills Table.csv

**Contenu :** Compétences de travail que possèdent les différents Pals :
- Minage
- Bûcheronnage
- Agriculture
- Construction
- Cuisine
- Etc.

**Rôle dans le jeu :** Ces compétences déterminent quelles tâches un Pal peut effectuer dans votre campement. Un Pal avec une compétence de minage élevée sera efficace pour extraire des minerais, tandis qu'un Pal avec des compétences agricoles sera utile pour cultiver des ressources alimentaires.

### 4.  Palworld_Data-Tower BOSS attribute comparison.csv

**Contenu :** Attributs des boss spéciaux qui apparaissent dans les tours du jeu.

**Rôle dans le jeu :** Les tours sont des défis endgame où les joueurs affrontent des boss puissants. Ces données aident à comprendre la puissance relative de ces boss et à préparer des stratégies adaptées pour les combattre.

### 5.  Palworld_Data-comparison of ordinary BOSS attributes.csv

**Contenu :** Attributs des boss ordinaires que l'on trouve dans le monde ouvert.

**Rôle dans le jeu :** Ces boss représentent des défis importants durant l'exploration et fournissent des récompenses précieuses. Comprendre leurs attributs permet aux joueurs de se préparer adéquatement à ces affrontements.

### 6.  Palworld_Data-hide pallu attributes.csv

**Contenu :** Informations sur des attributs cachés des Pals qui ne sont pas immédiatement visibles dans le jeu.

**Rôle dans le jeu :** Ces attributs cachés peuvent inclure des taux de croissance spécifiques, des affinités particulières, ou des capacités spéciales qui se débloquent dans certaines conditions. Ces informations sont précieuses pour les joueurs qui cherchent à optimiser leurs Pals.

---
## Implications pour l'Analyse

Cette structure de données nous permettra d'analyser :
- **Optimisation des équipes de combat** via les attributs de combat
- **Gestion efficace des ressources** via les compétences de travail
- **Stratégies de capture** via les zones d'apparition et probabilités
- **Progression du jeu** via les niveaux et raretés des Pals

## Import des bibliothèques Python de l'étude

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mariadb
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
import os
from pathlib import Path

## Chargement des données brutes

In [21]:
# Définition du chemin vers les fichiers fournis
data_path = "../database/"

# Vérification de l'existence du repertoire , if not utiliser le repertoire courrant
if not os.path.exists(data_path):
    data_path = "./"

# Affichage des fichiers dispo pour la vérif
files = [f for f in os.listdir(data_path) if f.endswith('.csv')] # ici on liste tous les fichiers ou dossiers + on itère chaque élément + Filtre pour garder que les fichier .csv
print("Fichiers dispo:")
for file in files: 
    print(f"-{file}")

# Chargement des données brutes & gestion d'erreurs grâce à pd.read_csv en définissant le chemin vers les fichiers.csv 
try:
    combat_attribute = pd.read_csv(f'{data_path}Palworld_Data--Palu combat attribute table.csv')
    refresh_area = pd.read_csv(f'{data_path}Palworld_Data--Palu refresh level.csv')
    ordinary_boss = pd.read_csv(f'{data_path}Palworld_Data-comparison of ordinary BOSS attributes.csv')
    tower_boss = pd.read_csv(f'{data_path}Palworld_Data-Tower BOSS attribute comparison.csv') 
    job_skill = pd.read_csv(f'{data_path}Palworld_Data-Palu Job Skills Table.csv')
    hidden_attribute = pd.read_csv(f'{data_path}Palworld_Data-hide pallu attributes.csv')
    print("\nDonnées brutes chargées")
except FileNotFoundError as e : 
  print(f"Erreur: {e}. Vérifier les noms des fichiers et le chemin.")

Fichiers dispo:
-Palworld_Data--Palu combat attribute table.csv
-Palworld_Data--Palu refresh level.csv
-Palworld_Data-comparison of ordinary BOSS attributes.csv
-Palworld_Data-hide pallu attributes.csv
-Palworld_Data-Palu Job Skills Table.csv
-Palworld_Data-Tower BOSS attribute comparison.csv

Données brutes chargées


## Création de la fonction d'analyse exploratoires des données fournies

In [27]:
def explore_dataframe(df, name):
    # Cette Fonction inclus un affichage optimisé
    
    # Sauvegarde et réglage des options d'affichage
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 30)
    pd.set_option('display.width', max(1200, df.shape[1] * 20))
    print(f"\n===== Exploration de {name} =====")
    
    # 1. Informations de base (similaire à df.info())
    print("\n--- Informations de base ---")
    print(f"Class: {type(df)}")
    # Index
    idx = df.index
    print(f"Index: {type(idx)} | {idx}")
    # Nombre total d'entrées et de colonnes
    print(f"Entries: {len(df)} | Columns: {df.shape[1]}")
    
    # Détail des colonnes et non-null counts
    structure = pd.DataFrame({
        'Column': df.columns,
        'Non-Null Count': df.notnull().sum().values,
        'Dtype': df.dtypes.values
    })
    display(structure)
    
    # Usage mémoire
    mem_usage = df.memory_usage(deep=True).sum()
    print(f"Memory usage: {mem_usage / 1024:.2f} KB")
    
    # 2. Dimensions
    print(f"\nDimensions: {df.shape[0]} lignes, {df.shape[1]} colonnes")
    
    # Aperçu des premières lignes
    print("\nAperçu des premières lignes:")
    display(df.head(3))
    
    # Types de données
    print("\nTypes de données:")
    display(df.dtypes)
    
    # Statistiques descriptives
    print("\nStatistiques descriptives:")
    display(df.describe(include='all'))
    
    # Valeurs manquantes
    print("\nValeurs manquantes:")
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_data = pd.concat([missing, missing_percent], axis=1, keys=['Total', 'Pourcentage'])
    display(missing_data[missing_data['Total'] > 0])
    if missing_data['Total'].sum() == 0:
        print("Aucune valeur manquante détectée")
    
    # Doublons
    duplicates = df.duplicated().sum()
    print(f"\nNombre de lignes dupliquées: {duplicates}")
    
    # Colonnes catégorielles
    print("\nAnalyse des colonnes catégorielles:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            unique_values = df[col].unique()
            n_unique = len(unique_values)
            
            print(f"\n{col}:")
            print(f"  - {n_unique} valeurs uniques")
            
            if n_unique <= 10:
                print(f"  - Valeurs: {unique_values}")
            elif n_unique <= 20:
                value_counts = df[col].value_counts().head(10)
                print(f"  - Top 10 des valeurs les plus fréquentes:")
                for val, count in value_counts.items():
                    print(f"    • {val}: {count} occurrences")
            else:
                value_counts = df[col].value_counts()
                print(f"  - Top 5 des valeurs les plus fréquentes:")
                for val, count in value_counts.head(5).items():
                    print(f"    • {val}: {count} occurrences")
                print(f"  - Valeur la moins fréquente: {value_counts.tail(1).index[0]} ({value_counts.tail(1).values[0]} occurrence(s))")
    else:
        print("Aucune colonne catégorielle détectée")
    
    # Rétablissement des options d'affichage
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_info_columns')
    pd.reset_option('display.max_colwidth')
    pd.reset_option('display.width')
    
    # Séparateur
    print("\n" + "="*70)

### Analyse exploratoire des données fournies

In [28]:
explore_dataframe(job_skill, 'Job Skills')


===== Exploration de Job Skills =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=139, step=1)
Entries: 139 | Columns: 23


,Column,Non-Null Count,Dtype
0,Related Links Directory Pa...,139,object
1,Unnamed: 1,139,object
2,Unnamed: 2,139,object
3,Unnamed: 3,139,object
4,Unnamed: 4,139,object
5,Unnamed: 5,26,object
6,Unnamed: 6,139,object
7,Unnamed: 7,139,object
8,Unnamed: 8,139,object
9,Unnamed: 9,139,object


Memory usage: 170.69 KB

Dimensions: 139 lignes, 23 colonnes

Aperçu des premières lignes:


,Related Links Directory Palu Combat Attribute Table Food BUFF Work Disease DEBUFF,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,ID,English name,Chinese name,Volume size,Food intake,night shift,Total skills,Make a fire,watering,planting,generate electricity,manual,collection,logging,Mining,pharmaceutical,cool down,pasture,carry,Handling speed,ranch items,pasture minimum output,The largest ranch (Rank = ...
1,1,Lamball,Mian Youyou,smallest,2,NaN,3,0,0,0,0,1,0,0,0,0,0,1,1,160,wool,1,Rank
2,2,Cattiva,Naughty cat,smallest,2,NaN,4,0,0,0,0,1,1,0,1,0,0,0,1,160,NaN,NaN,NaN



Types de données:


...


Statistiques descriptives:


,Related Links Directory Palu Combat Attribute Table Food BUFF Work Disease DEBUFF,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
count,139,139,139,139,139,26,139,139,139,139,139,139,139,139,139,139,139,139,139,57,14,14,14
unique,112,139,138,6,10,2,13,6,6,6,6,6,6,5,6,5,6,3,6,32,11,5,5
top,12,English name,Thunderbird,big,3,yes,3,0,0,0,0,0,0,0,0,0,0,0,0,250,wool,1,Rank
freq,2,1,2,43,26,25,27,116,123,117,127,87,92,102,115,122,120,125,82,6,3,9,9



Valeurs manquantes:


,Total,Pourcentage
Unnamed: 5,113,81.294964
Unnamed: 19,82,58.992806
Unnamed: 20,125,89.928058
Unnamed: 21,125,89.928058
Unnamed: 22,125,89.928058



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

Related Links Directory Palu Combat Attribute Table Food BUFF Work Disease DEBUFF:
  - 112 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • 12: 2 occurrences
    • 13: 2 occurrences
    • 37: 2 occurrences
    • 40: 2 occurrences
    • 45: 2 occurrences
  - Valeur la moins fréquente: 111 (1 occurrence(s))

Unnamed: 1:
  - 139 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • English name: 1 occurrences
    • Lamball: 1 occurrences
    • Cattiva: 1 occurrences
    • Chikipi: 1 occurrences
    • Lifmunk: 1 occurrences
  - Valeur la moins fréquente: Jetragon (1 occurrence(s))

Unnamed: 2:
  - 138 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • Chinese name: 1 occurrences
    • Naughty cat: 1 occurrences
    • Pipi Chicken: 1 occurrences
    • green leaf rat: 1 occurrences
  - Valeur la moins fréquente: Vortex Dragon (1 occurrence(s))

Unnam

In [25]:
explore_dataframe(refresh_area, 'Refresh area')


===== Exploration de Refresh area =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=149, step=1)
Entries: 149 | Columns: 19


,Column,Non-Null Count,Dtype
0,PS: This table only refres...,138,object
1,Unnamed: 1,137,object
2,Unnamed: 2,137,object
3,Unnamed: 3,137,object
4,Unnamed: 4,0,float64
5,Unnamed: 5,147,object
6,Unnamed: 6,146,object
7,Unnamed: 7,146,object
8,Unnamed: 8,146,object
9,Unnamed: 9,146,object


Memory usage: 146.81 KB

Dimensions: 149 lignes, 19 colonnes

Aperçu des premières lignes:


,"PS: This table only refreshes the data and does not include all occurrences of Palu. For example, the level 10 Balrog Sheep and Mammoth fights that appear in Maple Leaf Forest are random events. pending upgrade",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pallu refresh level reference,NaN,NaN,NaN,NaN,Minimum appearance level o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Palu's maximum appearance ...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Types de données:


...


Statistiques descriptives:


,"PS: This table only refreshes the data and does not include all occurrences of Palu. For example, the level 10 Balrog Sheep and Mammoth fights that appear in Maple Leaf Forest are random events. pending upgrade",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
count,138,137,137,137,0.0,147,146,146,146,146,18,146,0.0,141,140,140,140,7,140
unique,113,136,40,31,NaN,113,136,40,136,6,2,11,NaN,113,136,31,6,2,10
top,12,Thunderbird,2,45,NaN,32,Marshmallow,2,895,Creeps,yes,grassland,NaN,85,surf duck,45,Random dungeon boss,yes,dungeon
freq,2,2,12,31,NaN,3,2,13,2,102,17,49,NaN,3,2,32,71,6,74
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
"PS: This table only refreshes the data and does not include all occurrences of Palu. For example, the level 10 Balrog Sheep and Mammoth fights that appear in Maple Leaf Forest are random events. pending upgrade",11,7.382550
Unnamed: 1,12,8.053691
Unnamed: 2,12,8.053691
Unnamed: 3,12,8.053691
Unnamed: 4,149,100.000000
Unnamed: 5,2,1.342282
Unnamed: 6,3,2.013423
Unnamed: 7,3,2.013423
Unnamed: 8,3,2.013423
Unnamed: 9,3,2.013423



Nombre de lignes dupliquées: 1

Analyse des colonnes catégorielles:

PS: This table only refreshes the data and does not include all occurrences of Palu. For example, the level 10 Balrog Sheep and Mammoth fights that appear in Maple Leaf Forest are random events. pending upgrade:
  - 114 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • 12: 2 occurrences
    • 13: 2 occurrences
    • 32: 2 occurrences
    • 37: 2 occurrences
    • 40: 2 occurrences
  - Valeur la moins fréquente: 111 (1 occurrence(s))

Unnamed: 1:
  - 137 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • name: 1 occurrences
    • Naughty cat: 1 occurrences
    • Mian Youyou: 1 occurrences
    • green leaf rat: 1 occurrences
  - Valeur la moins fréquente: Vortex Dragon (1 occurrence(s))

Unnamed: 2:
  - 41 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • 2: 12 occurrences
    • 40: 10 occurrences
    • 18: 8 occurrences
    • 3: 7 occurrences

In [26]:
explore_dataframe(ordinary_boss, 'Ordinary Boss')


===== Exploration de Ordinary Boss =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=117, step=1)
Entries: 117 | Columns: 8


,Column,Non-Null Count,Dtype
0,The BOSS version of each P...,12,object
1,Unnamed: 1,11,object
2,Unnamed: 2,0,float64
3,Unnamed: 3,7,object
4,Unnamed: 4,7,object
5,Unnamed: 5,0,float64
6,Unnamed: 6,115,object
7,Unnamed: 7,115,object


Memory usage: 32.91 KB

Dimensions: 117 lignes, 8 colonnes

Aperçu des premières lignes:


,"The BOSS version of each Palu only takes 20%~40% damage, has a capture rate of about 70%, has a riding sprint speed of 100 points, and is larger in size.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,"In addition, a small numbe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,name,HP,NaN,name,Remote attack,NaN,name,Riding speed (BOSS is 100 ...



Types de données:


...


Statistiques descriptives:


,"The BOSS version of each Palu only takes 20%~40% damage, has a capture rate of about 70%, has a riding sprint speed of 100 points, and is larger in size.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
count,12,11,0.0,7,7,0.0,115,115
unique,12,7,NaN,7,5,NaN,113,20
top,"In addition, a small numbe...",260,NaN,name,90,NaN,Thunderbird,800
freq,1,2,NaN,1,2,NaN,2,21
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
"The BOSS version of each Palu only takes 20%~40% damage, has a capture rate of about 70%, has a riding sprint speed of 100 points, and is larger in size.",105,89.743590
Unnamed: 1,106,90.598291
Unnamed: 2,117,100.000000
Unnamed: 3,110,94.017094
Unnamed: 4,110,94.017094
Unnamed: 5,117,100.000000
Unnamed: 6,2,1.709402
Unnamed: 7,2,1.709402



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

The BOSS version of each Palu only takes 20%~40% damage, has a capture rate of about 70%, has a riding sprint speed of 100 points, and is larger in size.:
  - 13 valeurs uniques
  - Top 10 des valeurs les plus fréquentes:
    • In addition, a small number of BOSS have differences in HP and attack, see the table below. There is no difference in other attributes except those listed below: 1 occurrences
    • name: 1 occurrences
    • Chaos Knight BOSS: 1 occurrences
    • Chaos Knight: 1 occurrences
    • Winter Caller Beast BOSS: 1 occurrences
    • Winter Caller: 1 occurrences
    • Night Caller BOSS: 1 occurrences
    • Night Caller: 1 occurrences
    • Vortex Dragon BOSS: 1 occurrences
    • Vortex Dragon: 1 occurrences

Unnamed: 1:
  - 8 valeurs uniques
  - Valeurs: [nan 'HP' '260' '130' '420' '140' '330' '110']

Unnamed: 3:
  - 8 valeurs uniques
  - Valeurs: [nan 'name' 'Melupa BOSS' 'Melupa' 'Volt Meow BOSS' 'Vo

In [10]:
explore_dataframe(tower_boss, 'Tower Boss')


===== Exploration de Tower Boss =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=16, step=1)
Entries: 16 | Columns: 11


,Column,Non-Null Count,Dtype
0,name,16,object
1,Victor & Heterogeneous Gri...,16,object
2,Heterogeneous Griffin,16,object
3,Zoe & Lightning Bear,16,object
4,Lightning Bear,16,object
5,Marcus & Horus,16,object
6,Horus,16,object
7,Lily & Lily Queen,16,object
8,lily queen,16,object
9,Axel & Pollux,16,object


Memory usage: 10.58 KB

Dimensions: 16 lignes, 11 colonnes

Aperçu des premières lignes:


,name,Victor & Heterogeneous Griffin,Heterogeneous Griffin,Zoe & Lightning Bear,Lightning Bear,Marcus & Horus,Horus,Lily & Lily Queen,lily queen,Axel & Pollux,Pollux
0,HP,8000,120,6000,105,6500,100,5500,110,6500,100
1,melee attack,130,130,100,120,100,100,100,100,100,100
2,Remote attack,200,120,100,100,120,105,110,110,140,130



Types de données:


...


Statistiques descriptives:


,name,Victor & Heterogeneous Griffin,Heterogeneous Griffin,Zoe & Lightning Bear,Lightning Bear,Marcus & Horus,Horus,Lily & Lily Queen,lily queen,Axel & Pollux,Pollux
count,16,16,16,16,16,16,16,16,16,16,16
unique,16,14,13,11,12,14,14,12,11,12,10
top,HP,TRUE,120,100,100,100,100,100,100,100,100
freq,1,2,2,5,4,2,2,4,4,4,6



Valeurs manquantes:


,Total,Pourcentage


Aucune valeur manquante détectée

Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 16 valeurs uniques
  - Top 10 des valeurs les plus fréquentes:
    • HP: 1 occurrences
    • melee attack: 1 occurrences
    • Remote attack: 1 occurrences
    • defense: 1 occurrences
    • Support: 1 occurrences
    • experience ratio: 1 occurrences
    • slow walking speed: 1 occurrences
    • walking speed: 1 occurrences
    • running speed: 1 occurrences
    • riding speed: 1 occurrences

Victor & Heterogeneous Griffin:
  - 14 valeurs uniques
  - Top 10 des valeurs les plus fréquentes:
    • TRUE: 2 occurrences
    • 80: 2 occurrences
    • 200: 1 occurrences
    • 220: 1 occurrences
    • 8000: 1 occurrences
    • 130: 1 occurrences
    • 30: 1 occurrences
    • 90: 1 occurrences
    • 1100: 1 occurrences
    • 850: 1 occurrences

Heterogeneous Griffin:
  - 13 valeurs uniques
  - Top 10 des valeurs les plus fréquentes:
    • 120: 2 occurrences
    • FALSE: 2 occurrence

In [47]:
explore_dataframe(combat_attribute, 'Combat Attribut')


===== Exploration de Combat Attribut =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=138, step=1)
Entries: 138 | Columns: 50


,Column,Non-Null Count,Dtype
0,ID,138,int64
1,Chinese name,138,object
2,Name,138,object
3,CodeName,138,object
4,OverrideNameTextID,1,object
5,NamePrefixID,0,float64
6,OverridePartnerSkillTextID,1,object
7,IsPal,138,bool
8,Tribe,138,object
9,BPClass,138,object


Memory usage: 249.28 KB

Dimensions: 138 lignes, 50 colonnes

Aperçu des premières lignes:


,ID,Chinese name,Name,CodeName,OverrideNameTextID,NamePrefixID,OverridePartnerSkillTextID,IsPal,Tribe,BPClass,variant,Volume size,rarity,Element 1,Element 2,GenusCategory,Organization,weapon,WeaponEquip,nocturnal,4D total,HP,melee attack,Remote attack,defense,support,Speed of work,Level 1,Level 20,Level 50,AIRResponse,AISightResponse,endurance,slow walking speed,walking speed,running speed,Riding sprint speed,(being) damage multiplier,catch rate,Experience multiplier,price,Must bring entry 1,Must bring entry 2,Numerical description,lv1,lv2,lv3,lv4,lv5,Skill description
0,1,Mian Youyou,Lamball,SheepBall,NaN,NaN,NaN,True,SheepBall,SheepBall,NaN,XS,1,generally,NaN,Humanoid,NaN,NaN,False,NaN,280,70,70,70,70,100,100,105-106,205-236,362-441,Friendly,NaN,100,23,40,400,NaN,2,"1,5",1,1000,NaN,NaN,Ranch skills,0,0,0,0,0,"When activated, it will tr..."
1,3,Pipi Chicken,Chikipi,ChickenPal,NaN,NaN,NaN,True,ChickenPal,ChickenPal,NaN,XS,1,generally,NaN,Bird,NaN,NaN,False,NaN,250,60,70,60,60,70,100,104-105,190-217,325-392,Friendly,NaN,100,50,50,375,NaN,2,"1,5",1,1000,NaN,NaN,Ranch skills,0,0,0,0,0,Assign it to Palu Ranch an...
2,4,green leaf rat,Lifmunk,Carbunclo,NaN,NaN,NaN,True,Carbunclo,Carbunclo,NaN,XS,1,Wood,NaN,Humanoid,NaN,NaN,False,NaN,285,75,70,70,70,100,100,105-106,205-236,362-441,Escape,NaN,100,20,40,400,NaN,1,"0,9",1,1010,NaN,NaN,Skill power,10,11,13,16,20,"After being activated, it ..."



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,138.0,NaN,NaN,NaN,57.826087,31.829361,1.0,31.25,58.5,85.75,111.0
Chinese name,138,137,Thunderbird,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,138,138,Lamball,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CodeName,138,138,SheepBall,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverrideNameTextID,1,1,PAL_NAME_PlantSlime,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NamePrefixID,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverridePartnerSkillTextID,1,1,PARTNERSKILL_PlantSlime,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IsPal,138,1,True,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tribe,138,137,PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BPClass,138,138,SheepBall,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
OverrideNameTextID,137,99.275362
NamePrefixID,138,100.000000
OverridePartnerSkillTextID,137,99.275362
variant,105,76.086957
Element 2,112,81.159420
Organization,138,100.000000
weapon,138,100.000000
nocturnal,113,81.884058
AISightResponse,138,100.000000
Riding sprint speed,77,55.797101



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

Chinese name:
  - 137 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • Pipi Chicken: 1 occurrences
    • green leaf rat: 1 occurrences
    • tinder fox: 1 occurrences
    • surf duck: 1 occurrences
  - Valeur la moins fréquente: Vortex Dragon (1 occurrence(s))

Name:
  - 138 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Lamball: 1 occurrences
    • Chikipi: 1 occurrences
    • Lifmunk: 1 occurrences
    • Foxparks: 1 occurrences
    • Fuack: 1 occurrences
  - Valeur la moins fréquente: Jetragon (1 occurrence(s))

CodeName:
  - 138 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • SheepBall: 1 occurrences
    • ChickenPal: 1 occurrences
    • Carbunclo: 1 occurrences
    • Kitsunebi: 1 occurrences
    • BluePlatypus: 1 occurrences
  - Valeur la moins fréquente: JetDragon (1 occurrence(s))

OverrideNameTextID:
  - 2 valeurs uniques
  - V

In [48]:
explore_dataframe(hidden_attribute, 'Hidden Attribute')


===== Exploration de Hidden Attribute =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=175, step=1)
Entries: 175 | Columns: 70


,Column,Non-Null Count,Dtype
0,name,175,object
1,OverrideNameTextID,160,object
2,NamePrefixID,160,object
3,OverridePartnerSkillTextID,160,object
4,IsPal,175,bool
...,...,...,...
65,pasture,175,int64
66,Passive skill 1,17,object
67,Passive skill 2,4,object
68,Passive skill 3,0,float64


Memory usage: 227.22 KB

Dimensions: 175 lignes, 70 colonnes

Aperçu des premières lignes:


,name,OverrideNameTextID,NamePrefixID,OverridePartnerSkillTextID,IsPal,Tribe,BPClass,Pictorial ID,ZukanIndexSuffix,Size,rarity,Element 1,Element 2,GenusCategory,Organization,weapon,WeaponEquip,HP,melee attack,Remote attack,defense,support,CraftSpeed,(being) damage multiplier,Capture probability,Experience multiplier,price,AIRResponse,AISightResponse,slow walking speed,walking speed,running speed,Riding sprint speed,Handling speed,IsBoss,IsTowerBoss,BattleBGM,IgnoreLeanBack,IgnoreBlowAway,MaxFullStomach,FullStomachDecreaseRate,FoodAmount,ViewingDistance,ViewingAngle,HearingRate,NooseTrap,Nocturnal,BiologicalGrade,Predator,Edible,endurance,Male probability,fecundity,Breathing fire,watering,planting,generate electricity,manual,collection,logging,Mining,OilExtraction (not shown in game),pharmaceutical,cool down,carry,pasture,Passive skill 1,Passive skill 2,Passive skill 3,Passive skill 4
0,BOSS_Alpaca,Alpaca,BOSS_NAME_Alpaca,PARTNERSKILL_Alpaca,True,Alpaca,BOSS_Alpaca,-1,NaN,XL,3,Normal,NaN,FourLegged,NaN,NaN,False,90,90,90,90,100,100,"0,4","0,7",10,3040,NotInterested,NaN,62,62,460,900,261,True,False,FieldBoss,False,False,225,1,3,25,90,1,False,False,9,False,True,100,50,890,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN
1,BOSS_AmaterasuWolf,AmaterasuWolf,BOSS_NAME_AmaterasuWolf,PARTNERSKILL_AmaterasuWolf,True,AmaterasuWolf,BOSS_AmaterasuWolf,-1,NaN,XL,6,Fire,NaN,FourLegged,NaN,NaN,False,100,70,115,100,110,100,"0,2","0,7",10,3170,Boss,NaN,80,130,700,1000,415,True,False,FieldBoss,False,False,300,1,4,25,90,1,False,False,9,True,True,100,50,830,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,BOSS_BadCatgirl,BadCatgirl,BOSS_NAME_BadCatgirl,PARTNERSKILL_BadCatgirl,True,BadCatgirl,BOSS_BadCatgirl,-1,NaN,XL,0,Dark,NaN,Humanoid,NaN,NaN,False,100,100,100,100,100,100,"0,2","0,7",10,4690,Boss,NaN,-1,-1,-1,-1,-1,True,False,FieldBoss,False,False,225,1,3,25,90,1,False,True,9,False,True,100,50,0,0,0,0,0,2,3,1,0,0,0,0,2,0,NaN,NaN,NaN,NaN



Types de données:


...


Statistiques descriptives:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,175,175,BOSS_Alpaca,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverrideNameTextID,160,159,PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NamePrefixID,160,159,BOSS_NAME_PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverridePartnerSkillTextID,160,159,PARTNERSKILL_PlantSlime,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IsPal,175,1,True,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
pasture,175.0,NaN,NaN,NaN,0.074286,0.262988,0.0,0.0,0.0,0.0,1.0
Passive skill 1,17,11,Legend,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Passive skill 2,4,4,ElementBoost_Dark_2_PAL,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Passive skill 3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
OverrideNameTextID,15,8.571429
NamePrefixID,15,8.571429
OverridePartnerSkillTextID,15,8.571429
ZukanIndexSuffix,175,100.000000
Element 2,145,82.857143
Organization,175,100.000000
weapon,175,100.000000
AISightResponse,175,100.000000
Passive skill 1,158,90.285714
Passive skill 2,171,97.714286



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 175 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • BOSS_Alpaca: 1 occurrences
    • BOSS_AmaterasuWolf: 1 occurrences
    • BOSS_BadCatgirl: 1 occurrences
    • BOSS_Baphomet: 1 occurrences
    • BOSS_Baphomet_Dark: 1 occurrences
  - Valeur la moins fréquente: BOSS_ThunderBird (1 occurrence(s))

OverrideNameTextID:
  - 160 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • PlantSlime: 2 occurrences
    • AmaterasuWolf: 1 occurrences
    • Alpaca: 1 occurrences
    • Baphomet: 1 occurrences
    • Baphomet_Dark: 1 occurrences
  - Valeur la moins fréquente: ThunderBird (1 occurrence(s))

NamePrefixID:
  - 160 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • BOSS_NAME_PlantSlime: 2 occurrences
    • BOSS_NAME_AmaterasuWolf: 1 occurrences
    • BOSS_NAME_Alpaca: 1 occurrences
    • BOSS_NAME_Baphomet: 1 occurrences
    • BOSS_NAME_Baphomet_Dark: 1 occurrences
  - V

In [ ]:
## Création de la fonction d'analyse exploratoires des données propres

In [12]:
# Définition du chemin vers les fichiers propres
data_path = "../data/"

# Vérification de l'existence du repertoire , if not utiliser le repertoire courrant
if not os.path.exists(data_path):
    data_path = "./"

# Affichage des fichiers dispo pour la vérif
files = [f for f in os.listdir(data_path) if f.endswith('.csv')] # ici on liste tous les fichiers ou dossiers + on itère chaque élément + Filtre pour garder que les fichier .csv
print("Fichiers dispo:")
for file in files: 
    print(f"-{file}")

# Chargement des données brutes & gestion d'erreurs grâce à pd.read_csv en définissant le chemin vers les fichiers.csv 
try:
    combat_attribute = pd.read_csv(f'{data_path}Palworld_Data--Palu combat attribute table.csv')
    refresh_area = pd.read_csv(f'{data_path}Palworld_Data--Palu refresh level.csv')
    ordinary_boss = pd.read_csv(f'{data_path}Palworld_Data-comparison of ordinary BOSS attributes.csv')
    tower_boss = pd.read_csv(f'{data_path}Palworld_Data-Tower BOSS attribute comparison.csv') 
    job_skill = pd.read_csv(f'{data_path}Palworld_Data-Palu Job Skills Table.csv')
    hidden_attribute = pd.read_csv(f'{data_path}Palworld_Data-hide pallu attributes.csv')
    print("\nDonnées brutes chargées")
except FileNotFoundError as e : 
  print(f"Erreur: {e}. Vérifier les noms des fichiers et le chemin.")

Fichiers dispo:
-Palworld_Data--Palu combat attribute table.csv
-Palworld_Data--Palu refresh level.csv
-Palworld_Data-comparison of ordinary BOSS attributes.csv
-Palworld_Data-hide pallu attributes.csv
-Palworld_Data-Palu Job Skills Table.csv
-Palworld_Data-Tower BOSS attribute comparison.csv

Données brutes chargées


In [13]:
explore_dataframe(job_skill, 'Job Skills')


===== Exploration de Job Skills =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=138, step=1)
Entries: 138 | Columns: 23


,Column,Non-Null Count,Dtype
0,ID,138,int64
1,English name,138,object
2,Chinese name,138,object
3,Volume size,138,object
4,Food intake,138,int64
5,night shift,138,bool
6,Total skills,138,int64
7,Make a fire,138,bool
8,watering,138,bool
9,planting,138,bool


Memory usage: 46.97 KB

Dimensions: 138 lignes, 23 colonnes

Aperçu des premières lignes:


,ID,English name,Chinese name,Volume size,Food intake,night shift,Total skills,Make a fire,watering,planting,generate electricity,manual,collection,logging,Mining,pharmaceutical,cool down,pasture,carry,Handling speed,ranch items,pasture minimum output,The largest ranch
0,4,Lifmunk,green leaf rat,smallest,1,False,5,False,False,True,False,True,True,True,False,True,False,False,False,NaN,NaN,NaN,NaN
1,5,Foxparks,tinder fox,smallest,2,False,1,True,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN
2,9,Rooby,Fire Deer,Small,3,False,1,True,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN



Types de données:


...


Statistiques descriptives:


,ID,English name,Chinese name,Volume size,Food intake,night shift,Total skills,Make a fire,watering,planting,generate electricity,manual,collection,logging,Mining,pharmaceutical,cool down,pasture,carry,Handling speed,ranch items,pasture minimum output,The largest ranch
count,138.000000,138,138,138,138.000000,138,138.000000,138,138,138,138,138,138,138,138,138,138,138,138,56.000000,13,13,13
unique,NaN,138,137,5,NaN,2,NaN,2,2,2,2,2,2,2,2,2,2,2,2,NaN,10,4,4
top,NaN,Lifmunk,Thunderbird,big,NaN,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,NaN,wool,1,Rank
freq,NaN,1,2,43,NaN,113,NaN,116,123,117,127,87,92,102,115,122,120,125,82,NaN,3,9,9
mean,57.826087,NaN,NaN,NaN,4.550725,NaN,4.188406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278.250000,NaN,NaN,NaN
std,31.829361,NaN,NaN,NaN,2.263878,NaN,2.703204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.833244,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN
25%,31.250000,NaN,NaN,NaN,3.000000,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.000000,NaN,NaN,NaN
50%,58.500000,NaN,NaN,NaN,5.000000,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270.000000,NaN,NaN,NaN
75%,85.750000,NaN,NaN,NaN,7.000000,NaN,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,327.500000,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage
Handling speed,82,59.42029
ranch items,125,90.57971
pasture minimum output,125,90.57971
The largest ranch,125,90.57971



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

English name:
  - 138 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Lifmunk: 1 occurrences
    • Foxparks: 1 occurrences
    • Rooby: 1 occurrences
    • Jolthog Cryst: 1 occurrences
    • Gumoss: 1 occurrences
  - Valeur la moins fréquente: Orserk (1 occurrence(s))

Chinese name:
  - 137 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • tinder fox: 1 occurrences
    • Fire Deer: 1 occurrences
    • ice agouti: 1 occurrences
    • Ye NiNi: 1 occurrences
  - Valeur la moins fréquente: Pollux (1 occurrence(s))

Volume size:
  - 5 valeurs uniques
  - Valeurs: ['smallest' 'Small' 'medium' 'big' 'maximum']

ranch items:
  - 11 valeurs uniques
  - Top 10 des valeurs les plus fréquentes:
    • wool: 3 occurrences
    • gold: 2 occurrences
    • Palu Ball Advanced Palu Ball Arrow Gold Coin: 1 occurrences
    • Egg: 1 occurrences
    • milk: 1 occurrences
    

In [14]:
explore_dataframe(refresh_area, 'Refresh area')


===== Exploration de Refresh area =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=136, step=1)
Entries: 136 | Columns: 8


,Column,Non-Null Count,Dtype
0,id,136,int64
1,name,136,object
2,minimum level,136,int64
3,maximum level,136,int64
4,fecondity,136,int64
5,pallu refresh type,136,object
6,night only,136,bool
7,refresh area,136,object


Memory usage: 30.92 KB

Dimensions: 136 lignes, 8 colonnes

Aperçu des premières lignes:


,id,name,minimum level,maximum level,fecondity,pallu refresh type,night only,refresh area
0,1,Mian Youyou,1,14,1470,Creeps,False,grassland
1,2,Naughty cat,1,13,1460,Creeps,False,grassland
2,3,Pipi Chicken,1,13,1500,Creeps,False,grassland



Types de données:


...


Statistiques descriptives:


,id,name,minimum level,maximum level,fecondity,pallu refresh type,night only,refresh area
count,136.000000,136,136.000000,136.000000,136.000000,136,136,136
unique,NaN,135,NaN,NaN,NaN,5,2,10
top,NaN,Thunderbird,NaN,NaN,NaN,Creeps,False,grassland
freq,NaN,2,NaN,NaN,NaN,98,119,49
mean,57.279412,NaN,20.823529,33.036765,865.639706,NaN,NaN,NaN
std,31.686121,NaN,14.480067,12.140784,420.953005,NaN,NaN,NaN
min,1.000000,NaN,1.000000,10.000000,10.000000,NaN,NaN,NaN
25%,31.000000,NaN,9.000000,22.000000,487.500000,NaN,NaN,NaN
50%,58.000000,NaN,18.000000,38.000000,897.500000,NaN,NaN,NaN
75%,85.250000,NaN,32.250000,45.000000,1240.000000,NaN,NaN,NaN



Valeurs manquantes:


,Total,Pourcentage


Aucune valeur manquante détectée

Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 135 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Thunderbird: 2 occurrences
    • Naughty cat: 1 occurrences
    • Mian Youyou: 1 occurrences
    • green leaf rat: 1 occurrences
    • tinder fox: 1 occurrences
  - Valeur la moins fréquente: Winter Caller (1 occurrence(s))

pallu refresh type:
  - 5 valeurs uniques
  - Valeurs: ['Creeps' 'Secret Domain BOSS' 'event' 'Random dungeon boss' 'Wild BOSS']

refresh area:
  - 10 valeurs uniques
  - Valeurs: ['grassland' 'islands' 'dungeon' 'Snow' 'forest' 'volcano' 'game reserve'
 'desert' 'forest snow' 'Random events']



In [15]:
explore_dataframe(ordinary_boss, 'Ordinary Boss')


===== Exploration de Ordinary Boss =====

--- Informations de base ---
Class: <class 'pandas.core.frame.DataFrame'>
Index: <class 'pandas.core.indexes.range.RangeIndex'> | RangeIndex(start=0, stop=129, step=1)
Entries: 129 | Columns: 4


,Column,Non-Null Count,Dtype
0,name,129,object
1,HP,26,float64
2,Remote attack,18,float64
3,Riding speed (BOSS is 100 ...,129,int64


Memory usage: 12.18 KB

Dimensions: 129 lignes, 4 colonnes

Aperçu des premières lignes:


,name,HP,Remote attack,Riding speed (BOSS is 100 higher)
0,Heterogeneous Griffin BOSS,NaN,NaN,1300
1,Heterogeneous Griffin,NaN,NaN,1200
2,Ye Panda BOSS,NaN,NaN,800



Types de données:


...


Statistiques descriptives:


,name,HP,Remote attack,Riding speed (BOSS is 100 higher)
count,129,26.000000,18.00000,129.000000
unique,119,NaN,NaN,NaN
top,Snow Mammoth BOSS,NaN,NaN,NaN
freq,2,NaN,NaN,NaN
mean,NaN,238.076923,82.50000,922.480620
std,NaN,121.128666,6.47393,248.665284
min,NaN,110.000000,75.00000,500.000000
25%,NaN,132.500000,75.00000,750.000000
50%,NaN,200.000000,82.50000,900.000000
75%,NaN,330.000000,90.00000,1100.000000



Valeurs manquantes:


,Total,Pourcentage
HP,103,79.844961
Remote attack,111,86.046512



Nombre de lignes dupliquées: 0

Analyse des colonnes catégorielles:

name:
  - 119 valeurs uniques
  - Top 5 des valeurs les plus fréquentes:
    • Snow Mammoth BOSS: 2 occurrences
    • Thunderbird: 2 occurrences
    • Thunderbird BOSS: 2 occurrences
    • snow mammoth: 2 occurrences
    • Melupa BOSS: 2 occurrences
  - Valeur la moins fréquente: Cloud sea deer (1 occurrence(s))

